# I propose a baseline of 
# CycleGAN + cspdarknet53 for image classification.


I use following materials.
* Cyclegan's code: https://github.com/Lornatang/CycleGAN-PyTorch (unofficial code) and
> @inproceedings{CycleGAN2017,
>   title={Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networkss},
>   author={Zhu, Jun-Yan and Park, Taesung and Isola, Phillip and Efros, Alexei A},
>   booktitle={Computer Vision (ICCV), 2017 IEEE International Conference on},
>   year={2017}
> }
* Tensorflow tutorial: https://www.tensorflow.org/tutorials/generative/cyclegan?hl=ko.

* timm pytorch image model with several kaggle notebook.

![img](https://www.tensorflow.org/tutorials/generative/images/horse2zebra_1.png/)

At first time, I hope CycleGan do something like this.

It is not working well now. AUC value oscilate in low value and not converge.

But it is a prototype. My aim in this project is to update this code.

In [ ]:
import os
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from glob import glob

import pickle
import functools
import itertools
import random

import json
import numpy as np
import pandas as pd
import random
import math

import cv2
from PIL import Image
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from skimage.draw import line_aa
from tqdm import tqdm

import timm
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torchvision import transforms

In [ ]:
train_df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
train_df['filepath'] = train_df.id.apply(lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy')
train_df_0 = train_df[train_df.target==0].copy().reset_index(drop=True)
train_df_1 = train_df[train_df.target==1].copy().reset_index(drop=True)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device('cpu')
    print("GPU not available, CPU used")

In [ ]:
class setiDataset(Dataset):
    def __init__(self, df_0, df_1, transform = None): 
        self.df_0 = df_0
        self.df_1 = df_1
        self.transform = transform
    
    def fileinfo(self, idx):
        return self.df_0.filepath.iloc[idx], self.df_1.filepath.iloc[idx]
    
    def loadfile(self, filepath):
        image = np.load(filepath)
        image = np.vstack([image[0],image[2],image[4]]).astype('float32')
        image = cv2.resize(image, dsize=(260, 320), interpolation=cv2.INTER_AREA).transpose(1,0)
        image = np.stack([image,image,image])
        return image
    
    
    def __getitem__(self, idx):
        filepath_0, filepath_1 = self.fileinfo(idx)
        image_0 = self.loadfile(filepath_0)
        image_1 = self.loadfile(filepath_1)
        
        if self.transform:
            image_0 = image_0.transpose(1,2,0)
            augmented = self.transform(image=image_0)
            image_0 = augmented['image']
            
            image_1 = image_1.transpose(1,2,0)
            augmented = self.transform(image=image_1)
            image_1 = augmented['image']
            return image_0, image_1, torch.tensor(0, dtype=torch.long), torch.tensor(1, dtype=torch.long)

        return  torch.tensor(image_0, dtype=torch.float), torch.tensor(image_1, dtype=torch.float), torch.tensor(0, dtype=torch.long), torch.tensor(1, dtype=torch.long)
    
    def __len__(self):
        return len(self.df_1)

In [ ]:
train_transform = A.Compose([
    A.GridDistortion(p=0.75),
    A.OneOf([
        A.VerticalFlip(p=1),
        A.HorizontalFlip(p=1)
    ], p=0.5),
    A.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.05, rotate_limit=0, p=0.75),
    A.pytorch.ToTensor()
])

The above code is for augmentation image.<br>
I apply agumentation on image using albumentations libarary.

In [ ]:
# for visualization, not training.
SD = setiDataset(train_df_0, train_df_1)
SD_aug = setiDataset(train_df_0, train_df_1, transform = train_transform)

In [ ]:
for i in range(2):
    a = np.random.randint(low=0, high= len(SD))
    b = np.random.randint(low=0, high= len(SD))
    image1, _, _, _ = SD[a]
    image2, _, _, _ = SD_aug[a]
    plt.figure(figsize=(20,10))
    plt.subplot(2,2,1)
    plt.title('Target 0 Image, Original', fontsize=12)
    plt.imshow(image1[0],aspect='auto')
    plt.subplot(2,2,2)
    plt.title('Target 0 Image, Augmented', fontsize=12)
    plt.imshow(image2[0],aspect='auto')
    plt.show()
print(f'target0. {image1.shape}')

for i in range(2):
    a = np.random.randint(low=0, high= len(SD))
    b = np.random.randint(low=0, high= len(SD))
    _, image1, _, _ = SD[a]
    _, image2, _, _ = SD_aug[a]
    _, image3, _, _ = SD_aug[b]
    plt.figure(figsize=(20,10))
    plt.subplot(2,2,1)
    plt.title('Target 1 Image, Original', fontsize=12)
    plt.imshow(image1[0],aspect='auto')
    plt.subplot(2,2,2)
    plt.title('Target 1 Image, Augmented', fontsize=12)
    plt.imshow(image2[0],aspect='auto')
    plt.show()
print(f'targe01. {image1.shape}')

Images for training change slightly by using augmentation.

As a baseline, I use a cspdarknet53. <br>
The baseline model will check if the image is a needle or not.

In [ ]:
class cspdarknet_baseline(nn.Module):
    def __init__(self,model_name = 'cspdarknet53', pretrained = False):
        super(cspdarknet_baseline,self).__init__()
        print(f'Model: {model_name}')
        self.backbone = timm.create_model(model_name, pretrained=pretrained)
        self.myfc_1 = nn.Linear(1000,1)

    def forward(self, x):
        x = self.backbone(x)
        x = self.myfc_1(x)
        return x

In [ ]:
model = cspdarknet_baseline()
model.load_state_dict(torch.load('../input/epoch5/epoch5.pt'))

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
image_0, image_1, target0, target1 = SD[1]
plt.figure(figsize=(20,10))
plt.subplot(2,2,1)
plt.imshow(image_0[0], aspect='auto')
image_0 = torch.unsqueeze(image_0, dim=0)
target0 = int(target0.item())
output = model(image_0).item()
output = sigmoid(output)
plt.title(f'Target {target0}, Model value: {round(output,4)}', fontsize=12)

plt.subplot(2,2,2)
plt.imshow(image_1[0], aspect='auto')
image_1 = torch.unsqueeze(image_1, dim=0)
target1 = int(target1.item())
output = model(image_1).item()
output = sigmoid(output)
plt.title(f'Target {target1}, Model value: {round(output,4)}', fontsize=12)
plt.show()
print(image_1.shape)

The above value in pictures are the prediction of baseline model. <br>
The baseline model initially does not work well predict.

The blow fake image generator model is from

https://github.com/Lornatang/CycleGAN-PyTorch
> @inproceedings{CycleGAN2017,
>   title={Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networkss},
>   author={Zhu, Jun-Yan and Park, Taesung and Isola, Phillip and Efros, Alexei A},
>   booktitle={Computer Vision (ICCV), 2017 IEEE International Conference on},
>   year={2017}
> }

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Initial convolution block
            nn.ReflectionPad2d(3),
            nn.Conv2d(3, 64, 7),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            # Downsampling
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True),

            # Residual blocks
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),

            # Upsampling
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            # Output layer
            nn.ReflectionPad2d(3),
            nn.Conv2d(64, 3, 7),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)


class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()

        self.res = nn.Sequential(nn.ReflectionPad2d(1),
                                 nn.Conv2d(in_channels, in_channels, 3),
                                 nn.InstanceNorm2d(in_channels),
                                 nn.ReLU(inplace=True),
                                 nn.ReflectionPad2d(1),
                                 nn.Conv2d(in_channels, in_channels, 3),
                                 nn.InstanceNorm2d(in_channels))

    def forward(self, x):
        return x + self.res(x)

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        torch.nn.init.normal_(m.weight, 1.0, 0.02)
        torch.nn.init.zeros_(m.bias)

The generator has multiple residual block and convolution kernel.

In [ ]:
Gen_0_t1 = Generator()
Gen_1_t0 = Generator()
Gen_0_t1.apply(weights_init)
Gen_1_t0.apply(weights_init)
print('Generator model is created')

The below is an image converted using a initial generator.

In [ ]:
fake_image_1 = Gen_0_t1(image_1)
fake_image_1 = fake_image_1.detach().numpy().reshape(3,260,320)
plt.figure(figsize=(20,10))
plt.subplot(2,2,1)
plt.imshow(image_1.reshape(3,260,320)[0], aspect='auto')
plt.subplot(2,2,2)
plt.imshow(fake_image_1[0], aspect='auto')
plt.show()

The shape of the input and output is the same. <br>
Something is destroyed now but the generator may trained during trainig step.

The following two training codes for the generator and the model, respectively.

In [ ]:
def trainGenerator(data_loader, model, Gen_0_t1, Gen_1_t0, optimizer_G, device):
    model.eval()
    Gen_0_t1.train()
    Gen_1_t0.train()
    for data in tqdm(data_loader, position=0, leave=True, desc='Training'):
        image_0, image_1, target_0, target_1 = data
        
        image_0 = image_0.to(device, dtype=torch.float)
        target_0 = target_0.to(device, dtype=torch.float)
        image_1 = image_1.to(device, dtype=torch.float)
        target_1 = target_1.to(device, dtype=torch.float)
        
        
        optimizer_G.zero_grad()
        
        # Automorphism (Identity) loss
        auto_image_1 = Gen_0_t1(image_1)
        loss_identity_1 = nn.L1Loss()(auto_image_1, image_1) * 5.0
        
        
        auto_image_0 = Gen_1_t0(image_0)
        loss_identity_0 = nn.L1Loss()(auto_image_0, image_0) * 5.0
        

        # Generating loss
        fake_image_1 = Gen_0_t1(image_0)
        model_expect_fake_1 = model(fake_image_1)
        model_expect_fake_1 = nn.Sigmoid()(model_expect_fake_1)
        loss_GAN_0_t1 = nn.MSELoss()(model_expect_fake_1, target_1.view(-1,1))
        
        
        fake_image_0 = Gen_1_t0(image_1)
        model_expect_fake_0 = model(fake_image_0)
        model_expect_fake_0 = nn.Sigmoid()(model_expect_fake_0)
        loss_GAN_1_t0 = nn.MSELoss()(model_expect_fake_0, target_0.view(-1,1))
        
        
        
        # Cycle loss
        recovered_image_0 = Gen_1_t0(fake_image_1)
        loss_cycle_010 = nn.L1Loss()(recovered_image_0, image_0) * 10.0
        

        recovered_image_1 = Gen_1_t0(fake_image_0)
        loss_cycle_101 = nn.L1Loss()(recovered_image_1, image_1) * 10.0


        total_errG = loss_identity_1 + loss_identity_0 + loss_GAN_0_t1 + loss_GAN_1_t0 + loss_cycle_010 + loss_cycle_101
        total_errG.backward()
        
        optimizer_G.step()

In [ ]:
def trainModel(data_loader, model, optimizer, device):
    model.train()
    for data in tqdm(data_loader, position=0, leave=True, desc='Training'):
        image_0, image_1, target_0, target_1 = data
        
        images = torch.cat([image_0, image_1], dim = 0)
        targets = torch.cat([target_0, target_1])
        images = images.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        
        optimizer.zero_grad()
    
        # normal loss
        output = model(images)
        loss = nn.BCEWithLogitsLoss()(output, targets.view(-1,1))
        
        loss.backward()
        optimizer.step()
        
def trainModel2(data_loader, model, Gen_0_t1, Gen_1_t0, optimizer, device):
    Gen_0_t1.eval()
    Gen_1_t0.eval()
    model.train()
    
    for data in tqdm(data_loader, position=0, leave=True, desc='Training'):
        image_0, image_1, target_0, target_1 = data
        
        image_1 = image_1.to(device, dtype=torch.float)
        target_1 = target_1.to(device, dtype=torch.float)
        
        optimizer.zero_grad()
    
        # Generating loss
        fake_image_0 = Gen_1_t0(image_1)
        model_expect_fake_0 = model(fake_image_0)
        model_expect_fake_0 = nn.Sigmoid()(model_expect_fake_0)
        loss_model_0 = nn.MSELoss()(model_expect_fake_0,  target_1.view(-1,1)) * 0.5
        
        image_0 = image_0.to(device, dtype=torch.float)
        target_0 = target_0.to(device, dtype=torch.float)
        fake_image_1 = Gen_0_t1(image_0)
        model_expect_fake_1 = model(fake_image_1)
        model_expect_fake_1 = nn.Sigmoid()(model_expect_fake_1)
        loss_model_1 = nn.MSELoss()(model_expect_fake_1,  target_0.view(-1,1)) * 0.5
        
        loss = loss_model_0 + loss_model_1
        loss.backward()
        optimizer.step()       

In [ ]:
def evaluate(data_loader, model, device):
    model.eval()
    
    final_targets = []
    final_outputs = []
    validate_losses = 0
    cnt = 0
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            cnt += 1
            image_0, image_1, target_0, target_1 = data

            images = torch.cat([image_0, image_1], dim = 0)
            targets = torch.cat([target_0, target_1])
            
            images = images.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            
            output = model(images)
            loss = nn.BCEWithLogitsLoss()(output, targets.view(-1, 1))
            validate_losses += loss.item()


            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            
            final_targets.extend(targets)
            final_outputs.extend(output)
    return final_outputs, final_targets, validate_losses/cnt

In [ ]:
import gc
try:
    obj = None
    gc.collect()
    torch.cuda.empty_cache()
    del model
    del SD_aug
    del SD
    del Gen_0_t1
    del Gen_1_t0
except:
    pass

I believe the loss from cyclegan act as a Regulrization of base model's decision boundary.


Since the code is not optimized, so the memory keeps fulled.

I use batch size 4...

In [ ]:
Batch_Size = 4

#cspdarknet53
model = cspdarknet_baseline(pretrained=True)
model.to(device)
optimizer1 = torch.optim.Adam(model.parameters(), lr=3e-4, eps=1e-5)
optimizer2 = torch.optim.RMSprop(model.parameters(), lr=5e-6, eps=1e-5)

#Image generator 0 to 1 and 1 to 0.
Gen_0_t1 = Generator()
Gen_1_t0 = Generator()
Gen_0_t1.apply(weights_init)
Gen_1_t0.apply(weights_init)

Gen_0_t1.to(device)
Gen_1_t0.to(device)
optimizer_G = torch.optim.Adam(itertools.chain(Gen_0_t1.parameters(), Gen_1_t0.parameters()),
                               lr=2e-4, betas=(0.5, 0.999))


train_df_1 = train_df[train_df.target==1].reset_index(drop=True)
train_df_0 = train_df[train_df.target==0].sample(len(train_df_1)).reset_index(drop=True)


skf = StratifiedKFold(n_splits=5)
X = train_df_1.filepath.values
Y = train_df_0.target.values

for train_index, test_index in skf.split(X, Y):
    
    tra1_df = train_df_1.loc[train_index]
    tra0_df = train_df_0.loc[train_index]
    val1_df = train_df_1.loc[test_index]
    val0_df = train_df_0.loc[test_index]
    
    train_dataset = setiDataset(tra0_df, tra1_df, transform = train_transform)
    train_loader= torch.utils.data.DataLoader(train_dataset, batch_size=Batch_Size, shuffle=False)
    valid_dataset = setiDataset(val0_df, val1_df)
    valid_loader= torch.utils.data.DataLoader(valid_dataset, batch_size=Batch_Size, shuffle=False)
    
    trainModel(train_loader, model, optimizer1, device)
    predictions, valid_targets, valid_loss = evaluate(valid_loader, model, device=device)
    roc_auc = metrics.roc_auc_score(valid_targets, predictions)
    print(f"Part1: Valid Loss={valid_loss}, Valid ROC AUC={roc_auc}")
    
    trainGenerator(train_loader,model, Gen_0_t1, Gen_1_t0, optimizer_G, device)
    trainModel2(train_loader, model, Gen_0_t1, Gen_1_t0, optimizer2, device)

    predictions, valid_targets, valid_loss = evaluate(valid_loader, model, device=device)
    roc_auc = metrics.roc_auc_score(valid_targets, predictions)
    print(f"Part2: Valid Loss={valid_loss}, Valid ROC AUC={roc_auc}")

I use validation set dependently with a traning set.<br>
So the performance has a strict upper bound of the score after 2 epoch.

I will fix this problem in near future.

In [ ]:
torch.save(model.state_dict(),'model.pt')
torch.save(Gen_0_t1.state_dict(),'Gen_0_t1.pt')
torch.save(Gen_1_t0.state_dict(),'Gen_1_t0.pt')  

Finally, I visualization training results for one data.

In [ ]:
SD = setiDataset(train_df_0, train_df_1)
image0, image1, t0, t1 = SD[1]

plt.figure(figsize=(20,10))
plt.subplot(2,2,1)
plt.imshow(image0[0], aspect='auto')
image0 = torch.unsqueeze(image0, dim=0)
image0 = image0.to('cuda')
output = model(image0).item()
output = sigmoid(output)
plt.title(f'Target {0}, Model value: {round(output,4)}', fontsize=12)

plt.subplot(2,2,2)
Gen_0_t1.eval()
fake_image_1 = Gen_0_t1(image0)
output2 = model(fake_image_1).item()
output2 = sigmoid(output2)
fake_image_1 = fake_image_1.detach().cpu().numpy()
fake_image_1 = fake_image_1.reshape(3,260,320)
plt.imshow(fake_image_1[0], aspect='auto')
plt.title(f'Target {0} to Fake 1, Model value: {round(output2,4)}', fontsize=12)
plt.show()

Something... hmm

In [ ]:
SD = setiDataset(train_df_0, train_df_1)
image0, image1, t0, t1 = SD[2]

plt.figure(figsize=(20,10))
plt.subplot(2,2,1)
plt.imshow(image0[0], aspect='auto')
image0 = torch.unsqueeze(image0, dim=0)
image0 = image0.to('cuda')
output = model(image0).item()
output = sigmoid(output)
plt.title(f'Target {0}, Model value: {round(output,4)}', fontsize=12)

plt.subplot(2,2,2)
Gen_0_t1.eval()
fake_image_1 = Gen_0_t1(image0)
output2 = model(fake_image_1).item()
output2 = sigmoid(output2)
fake_image_1 = fake_image_1.detach().cpu().numpy()
fake_image_1 = fake_image_1.reshape(3,260,320)
plt.imshow(fake_image_1[0], aspect='auto')
plt.title(f'Target {0} to Fake 1, Model value: {round(output2,4)}', fontsize=12)
plt.show()

???????